In [3]:
# 安裝必要套件（在 Colab 中通常已內建 SciPy 與 NumPy，可視情況拿掉）
# !pip install scipy numpy

import numpy as np
from scipy import optimize as opt

# 參數：票價、座位數、顧客出現機率、賠償金
PRICE = 150
SEATS = 19
P_SHOW = 0.9
COMP_COST = 325

# 講義提供的經驗需求分佈
DEMAND_VALUES = np.arange(14, 26)          # 14 到 25
DEMAND_PROBS  = np.array([0.03, 0.05, 0.07, 0.09, 0.11,
                          0.15, 0.18, 0.14, 0.08, 0.05,
                          0.03, 0.02])

# 單次評估函式：回傳平均利潤、超賣機率、平均 lost sales
def evaluate_q(q, S=50000, seed=2025):
    rng = np.random.default_rng(seed)
    demand = rng.choice(DEMAND_VALUES, size=S, p=DEMAND_PROBS)
    reserved = np.minimum(demand, q)
    # 依 reserved 數目進行二項抽樣決定實際報到人數
    show = rng.binomial(reserved, P_SHOW)
    # 賠償金只會在 show > 座位數時發生
    compensation = np.clip(show - SEATS, 0, None) * COMP_COST
    profit = PRICE * reserved - compensation
    bump_prob = np.mean(show > SEATS)
    lost_sales = np.mean(np.clip(demand - q, 0, None))
    return profit.mean(), bump_prob, lost_sales

# 搜尋符合「超賣機率 ≤ 5 %」且平均利潤最高的 Q
best_q, best_profit, best_bump, best_lost = None, -1e9, None, None
for q in range(14, 31):            # 探索 14~30 張預約
    avg_profit, bump_prob, lost_sales = evaluate_q(q)
    if bump_prob <= 0.05 and avg_profit > best_profit:
        best_q, best_profit = q, avg_profit
        best_bump, best_lost = bump_prob, lost_sales

print(f"最佳預訂量 Q = {best_q}")
print(f"平均利潤 ≈ ${best_profit:,.0f}")
print(f"超賣機率 ≈ {best_bump:.2%}")
print(f"預期 lost sales ≈ {best_lost:.2f} 位乘客")


最佳預訂量 Q = 19
平均利潤 ≈ $2,721
超賣機率 ≈ 0.00%
預期 lost sales ≈ 1.15 位乘客
